# Backend : iteraction with user

In [101]:
# import
import os
import folium
import pyproj
import pandas as pd
import geopandas as gpd
import plotly_express as px
from datetime import datetime, date
import openrouteservice as ors
import matplotlib.pyplot as plt
from shapely.ops import transform
from IPython.display import display
from shapely.geometry import Point, LineString, Polygon

### User input coordinates : addresses later.

In [102]:
# input coordinates
coordinates = [[2.3668617010116577,48.86100925394748 ], [2.4125343561172485, 48.83392954811057]]

In [103]:
# openrouteservices 
client = ors.Client(key='5b3ce3597851110001cf62485169f9e2d05c419eaf0607fe2b5b0bfa')

#requete openrouteservice
route = client.directions(
    coordinates=coordinates,
    profile='cycling-regular',
    format ='geojson',
    units ='km'
)

route

ConnectionError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /v2/directions/cycling-regular/geojson (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc706c4cbe0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
# get clusters
cluster_polygons_file = "../model/model_cluster/cluster_polygons.geojson"
gdf_cluster = gpd.read_file(cluster_polygons_file)
display(gdf_cluster.head(2))

# clusters with buffer
orig_crs = 4326
fr_crs = 2154
gdf_cluster = gdf_cluster.to_crs(epsg=fr_crs)

# crs
buffer_m = 15
gdf_cluster['geometry'] = gdf_cluster.geometry.buffer(buffer_m)
#gdf_cluster = gdf_cluster.buffer(buffer_m)

# set crs
gdf_cluster = gdf_cluster.to_crs(epsg=orig_crs)

type(gdf_cluster)
gdf_cluster

,cluster_index,geometry
0,0,"POLYGON ((2.41149 48.84703, 2.41106 48.84705, ..."
1,1,"POLYGON ((2.36897 48.85400, 2.36893 48.85403, ..."


,cluster_index,geometry
0,0,"POLYGON ((2.41169 48.84707, 2.41170 48.84705, ..."
1,1,"POLYGON ((2.36917 48.85403, 2.36917 48.85401, ..."
2,2,"POLYGON ((2.37251 48.84846, 2.37249 48.84846, ..."
3,3,"POLYGON ((2.31386 48.86467, 2.31386 48.86465, ..."
4,4,"POLYGON ((2.29898 48.86840, 2.29897 48.86839, ..."
...,...,...
139,139,"POLYGON ((2.37868 48.85880, 2.37869 48.85879, ..."
140,140,"POLYGON ((2.36493 48.85404, 2.36495 48.85404, ..."
141,141,"POLYGON ((2.35992 48.87225, 2.35993 48.87223, ..."
142,142,"POLYGON ((2.37064 48.89819, 2.37062 48.89819, ..."


In [ ]:
# user route to df
user_line_coords = route['features'][0]['geometry']['coordinates']
user_line_df = pd.DataFrame(user_line_coords,
                            columns=["lon", "lat"])

# df to gdf
user_line_gdf = gpd.GeoDataFrame(user_line_df, 
                    geometry=gpd.points_from_xy(user_line_df.lon, user_line_df.lat),
                    crs="epsg:4326")

display(user_line_gdf.sample(2))

,lon,lat,geometry
43,2.376768,48.859725,POINT (2.37677 48.85972)
45,2.378509,48.858679,POINT (2.37851 48.85868)


In [ ]:
# intersect : spatial join
cluster_on_route = gpd.sjoin(gdf_cluster, user_line_gdf)[gdf_cluster.columns]
display(cluster_on_route.sample(2))

,cluster_index,geometry
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."
18,18,"POLYGON ((2.40662 48.83522, 2.40660 48.83522, ..."


In [ ]:
# localisation Paris
m = folium.Map(location=[48.862, 2.346], tiles='cartodbpositron', zoom_start=12)

# # cluster on map
# gdf = gdf_cluster
# for _, r in gdf.iterrows():
#     # Without simplifying the representation of each borough,
#     # the map might not be displayed
#     sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: 
#                            {'fillColor': '#669bbc',
#                             'color': '#669bbc'}
#     )
    
#     folium.Popup(r['cluster_index']).add_to(geo_j)
#     geo_j.add_to(m)
    
# cluster intersection map
gdf = cluster_on_route
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: 
                           {'fillColor': '#eb353c',
                            'color': '#eb353c',
                            'weight':5,
                            'stroke':False}
    )
    
    folium.Popup(r['cluster_index']).add_to(geo_j)
    geo_j.add_to(m)
    
# add user itinerary polyline
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in user_line_coords],
                           color = '#84dcc6',
                           smooth_factor=0.9,
                           weight = 1.5

).add_to(m)

In [ ]:
m

In [ ]:
# export map
m.save("./templates/map.html")

In [ ]:
## GEOFENCING
# add time field in user route 
pip_mask_geofence = user_line_gdf.intersects(gdf_cluster.unary_union)

# geofence column
user_line_gdf["geofence"] = pip_mask_geofence

# add time
number_of_points = user_line_gdf.shape[0]
start_date = datetime(2021, 11, 22, 12, 10)
date_range = pd.date_range(start=start_date, periods=number_of_points)

# time column
if not "time" in user_line_gdf.columns : 
    user_line_gdf["time"] = date_range

# replace gepfencing by In and Out
user_line_gdf["geofence"] = user_line_gdf["geofence"].replace({True: "In", False: "Out"})

user_line_gdf


,lon,lat,geometry,geofence,time
0,2.366860,48.861006,POINT (2.36686 48.86101),Out,2021-11-24 15:25:51.516886
1,2.364678,48.861313,POINT (2.36468 48.86131),Out,2021-11-25 15:25:51.516886
2,2.364594,48.861326,POINT (2.36459 48.86133),Out,2021-11-26 15:25:51.516886
3,2.364598,48.861389,POINT (2.36460 48.86139),Out,2021-11-27 15:25:51.516886
4,2.364603,48.861487,POINT (2.36460 48.86149),Out,2021-11-28 15:25:51.516886
...,...,...,...,...,...
190,2.412759,48.833494,POINT (2.41276 48.83349),Out,2022-06-02 15:25:51.516886
191,2.412849,48.833551,POINT (2.41285 48.83355),Out,2022-06-03 15:25:51.516886
192,2.412773,48.833630,POINT (2.41277 48.83363),Out,2022-06-04 15:25:51.516886
193,2.412632,48.833771,POINT (2.41263 48.83377),Out,2022-06-05 15:25:51.516886


In [ ]:
# geofencing
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
fig = px.scatter_mapbox(
    user_line_gdf, 
    lat="lat", 
    lon="lon" ,
    size_max=6, 
    zoom=8, width=1200, height=800)
fig.show(renderer="iframe")